In [149]:
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
import random
import pygame
import math
import torch
from torch import optim
from tensorboardX import SummaryWriter
import time
import load
from models import EncoderRNN, DecoderRNN, NeighbourhoodFullyConnected, NeighbourhoodFullyConnectedDecoder, SimpleEnd2End
import load_topology_dataset
from load_topology_dataset import MAX_CELL_COUNT, MIN_CELL_COUNT, IMAGE_SIZE
device = "cpu"
writer = SummaryWriter('runs/exp-1')
writer3 = SummaryWriter(comment='3x learning rate')

# Simplifying Assumptions
1. No multiple carriers.
2. Locations of loads and cells discretized to grid of resolution IMAGE_SIZE
3. Cells are not limited by capacity
4. Distribution of load follows inverse square of distance.
5. Cells are omnidirectional.
6. Load is constant.
7. Only a maximum of 1 cell changes (addition or removal) at a time.

In [2]:
test_loads = ((1,12), (22, 22))
test_loads = [(1,12)]
test_cells = [(2, 12), (31, 0)]
load.calculate_cell_load(cells=test_cells, loads=test_loads)

[(2, 12, 0.6709275647328745), (31, 0, 0.3290724352671256)]

# Early Fusion

# Visualisation

In [3]:
PIXELS = 640
LEFT = 1
RIGHT = 3
INPUT_SCREEN_COUNT = 1
TARGET_SCREEN_COUNT = 1
SCREEN_COUNT = INPUT_SCREEN_COUNT + TARGET_SCREEN_COUNT

PIXELS_PER_BLOCK = int(PIXELS / IMAGE_SIZE)
SCALE_ARR = np.ones((PIXELS_PER_BLOCK, PIXELS_PER_BLOCK, 1))
print(PIXELS_PER_BLOCK)

load_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 1))
load_screen[1,12, 0] = 255
load_screen[22,22, 0] = 255
loads = [(1, 12), (22, 22)]
block_img = np.kron(load_screen, SCALE_ARR)

screen = pygame.display.set_mode((PIXELS, PIXELS))
screens = []

#for i in range(SCREEN_COUNT):
##    screen_draw = pygame.Surface((PIXELS, PIXELS))
#    screens.append(screen_draw)


screen_draw_idx = 0


draw_on = False
last_pos = (0, 0)
color = (255, 128, 0)
radius = 10
device = "cpu"

block_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))

model_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)
frame = 0


20


In [4]:
a = ["abc", "def", "ghi", "jkl"]

In [5]:
a[2:]

['ghi', 'jkl']

In [6]:
def add_cell_if_not_exists(new_cell, cells):
    cell_already_exists = False

    for i, cell in enumerate(cells):
        if cell[0] == new_cell[0] and cell[1] == new_cell[1]:
            cell_already_exists = True
            break
    if not cell_already_exists:
        cells.append(new_cell)
    return cells

def remove_cell_if_exists(remove_cell, cells):
    for i, cell in enumerate(cells):
        if cell[0] == remove_cell[0] and cell[1] == remove_cell[1]:
            cells.pop(i)
            break
    return cells

# Visualisation

In [11]:
screen_draw_idx = 0
cells_seq = [list() for i in range(SCREEN_COUNT)]
cells_load_seq = [list() for i in range(SCREEN_COUNT)]
BATCH_SIZE=1
show_labels = True

while True:
    
    ##############################
    # Use model to forecast future load
    ##############################
    load_cells_seq_input = cells_load_seq[:INPUT_SCREEN_COUNT]
    load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
    
    model_target = [list() for i in range(TARGET_SCREEN_COUNT)]
    for reference_cell in cells_seq[-1]:
        #print(reference_cell)
        ref_x, ref_y = reference_cell[0], reference_cell[1]
        reference_cell_input = np.zeros((INPUT_SCREEN_COUNT))
        reference_cell_present_input = np.zeros((INPUT_SCREEN_COUNT))
        neighbourhood_cell_rel_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT, 2))
        neighbourhood_cell_load_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT))
        neighbourhood_cell_present_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT))
        for seq_idx in range(INPUT_SCREEN_COUNT):
            reference_cell_active = False
            cell_idx = 0
            for cell in load_cells_seq_input[seq_idx]:
                current_x = cell[0]
                current_y = cell[1]
                current_load = cell[2]
                # If not reference cell add to neighbourhood
                if not (current_x == ref_x and current_y == ref_y):
                    # neighbourhood normalized by dividing by IMAGE_SIZE.
                    # Subtract from 1 since cells that are closes have greated influence.
                    #print("seq_idx {}".format(seq_idx))
                    #print("cell_idx {}".format(cell_idx))
                    neighbourhood_cell_rel_input[seq_idx, cell_idx, 0] = 1. - (ref_x - current_x) / IMAGE_SIZE
                    neighbourhood_cell_rel_input[seq_idx, cell_idx, 1] = 1. - (ref_y - current_y) / IMAGE_SIZE
                    neighbourhood_cell_load_input[seq_idx, cell_idx] = current_load
                    cell_idx += 1
                else:
                    # print("ref cell")
                    reference_cell_input[seq_idx] = current_load
                    reference_cell_active = True
            if reference_cell_active:
                reference_cell_present_input[seq_idx] = 1.0

        load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
        reference_cell_present_target = np.zeros((TARGET_SCREEN_COUNT))
        neighbourhood_cell_rel_target = np.zeros((TARGET_SCREEN_COUNT, MAX_CELL_COUNT, 2))


        for seq_idx in range(TARGET_SCREEN_COUNT):
            reference_cell_active = False
            cell_idx = 0
            for cell in load_cells_seq_target[seq_idx]:
                current_x = cell[0]
                current_y = cell[1]
                current_load = cell[2]
                #print(current_load)
                # If not reference cell add to neighbourhood
                if not (current_x == ref_x and current_y == ref_y):
                    # neighbourhood normalized by dividing by IMAGE_SIZE.
                    # Subtract from 1 since cells that are closes have greated influence.
                    neighbourhood_cell_rel_target[seq_idx, cell_idx, 0] = 1. - abs(ref_x - current_x) / IMAGE_SIZE
                    neighbourhood_cell_rel_target[seq_idx, cell_idx, 1] = 1. - abs(ref_y - current_y) / IMAGE_SIZE
                    cell_idx += 1
                else:
                    reference_cell_active = True
            if reference_cell_active:
                reference_cell_present_target[seq_idx] = 1.0

        reference_cell_input = torch.tensor(reference_cell_input, dtype=torch.float32).unsqueeze(0)
        reference_cell_present_input = torch.tensor(reference_cell_present_input, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_rel_input = torch.tensor(neighbourhood_cell_rel_input, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_load_input = torch.tensor(neighbourhood_cell_load_input, dtype=torch.float32).unsqueeze(0)
        reference_cell_present_target = torch.tensor(reference_cell_present_target, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_rel_target = torch.tensor(neighbourhood_cell_rel_target, dtype=torch.float32).unsqueeze(0)

        for seq_idx in range(INPUT_SCREEN_COUNT):
            neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)
            for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
                output = neighbourhood_fully_connected(neighbourhood_cell_rel_input[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2),
                                              neighbourhood_cell_load_input[:,seq_idx, neighbourhood_cell_idx].view(BATCH_SIZE, 1))
                neighbourhood_influence = torch.add(neighbourhood_influence, output)
            # Concatonate neighbourhood with reference cell time series.
            cat_ref_input = (torch.cat((reference_cell_input[:, seq_idx].view(BATCH_SIZE, 1),
                                                reference_cell_present_input[:, seq_idx].view(BATCH_SIZE, 1)), dim=1))
            encoder_input = torch.cat((cat_ref_input , neighbourhood_influence), dim=1).view(BATCH_SIZE, 1, 1 + 1+ 16)
            if seq_idx==0:
                encoder_hidden = torch.zeros(2, BATCH_SIZE, HIDDEN_SIZE, device=device)
            encoder_output, encoder_hidden = encoder(encoder_input, encoder_hidden)
        #print("encoder_hidden {}".format(encoder_hidden))

        for seq_idx in range(TARGET_SCREEN_COUNT):
            neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)

            #print("reference_cell_input {}".format(reference_cell_input))
            
            for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
                output = neighbourhood_fully_connected_decoder(neighbourhood_cell_rel_target[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2))
                neighbourhood_influence = torch.add(neighbourhood_influence, output)

            #print("output {}".format(output))
            #print("neighbourhood_influence {}".format(neighbourhood_influence))
            if seq_idx==0:
                decoder_output, decoder_hidden = decoder(reference_cell_input[:, -1].view(BATCH_SIZE, 1),
                                                         reference_cell_present_input[:, -1].view(BATCH_SIZE, 1),
                                                         neighbourhood_influence.view(BATCH_SIZE, 16),
                                                         encoder_hidden)
                loss = criterion(decoder_output, reference_cell_target[:, seq_idx])

                decoder_output, decoder_hidden = decoder(decoder_output.view(BATCH_SIZE, 1),
                                                         reference_cell_present_target[:, seq_idx -1].view(BATCH_SIZE, 1),
                                                         neighbourhood_influence.view(BATCH_SIZE, 16),
                                                         decoder_hidden)
            model_target[seq_idx].append((ref_x, ref_y, decoder_output.detach().numpy()[0][0][0]))
            
        
        #print("decoder_target {}".format(decoder_output))
        #print("model_target {}".format(model_target))
    #print(decoder_output)
    ################################
    # DISPLAY
    #################################
    screen_draw = pygame.Surface((PIXELS, PIXELS))
    
    #print(cells_seq)
    cells = cells_seq[screen_draw_idx]
    cell_loads = cells_load_seq[screen_draw_idx]

    if (screen_draw_idx < INPUT_SCREEN_COUNT) or (show_labels and screen_draw_idx >= INPUT_SCREEN_COUNT):
        for cell in cell_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )
    else:
        model_loads = model_target[screen_draw_idx - INPUT_SCREEN_COUNT]
        for cell in model_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )

    e = pygame.event.poll()


    if e.type == pygame.QUIT:
        raise StopIteration
    if e.type == pygame.MOUSEBUTTONDOWN:
        leftclick, middleclick, rightclick = pygame.mouse.get_pressed()
        x, y = e.pos
        x_cell= int(x / PIXELS_PER_BLOCK)
        y_cell = int(y / PIXELS_PER_BLOCK)
        if leftclick:
            cells = add_cell_if_not_exists((x_cell, y_cell), cells)
        elif rightclick:
            cells = remove_cell_if_exists((x_cell, y_cell), cells)            
        draw_on = True
        # Update cells
        cells_seq[screen_draw_idx] = cells
        
        # Update all cell loads
        for i, cells in enumerate(cells_seq):
            cells_load_seq[i] = load.calculate_cell_load(cells, loads)
    
    if e.type == pygame.KEYDOWN:
        if e.key == pygame.K_SPACE:
            if show_labels:
                show_labels = False
            else:
                show_labels = True
                print(model_target[-1])
                print(load_cells_seq_target[-1])
        if e.key ==  pygame.K_LEFT:
            if screen_draw_idx > 0:
                screen_draw_idx -= 1
        if e.key ==  pygame.K_RIGHT:
            if screen_draw_idx < SCREEN_COUNT - 1:
                screen_draw_idx += 1
    if e.type == pygame.MOUSEBUTTONUP:
        draw_on = False
    pygame.display.flip()
    target_screen = pygame.surfarray.array3d(screen_draw)
    display_img = block_img + target_screen
    new_surf = pygame.pixelcopy.make_surface(display_img.astype(np.uint8))
    screen.blit(new_surf, (0, 0))

[(8, 7, 2.2508202)]
[(8, 7, 2.0)]
[(8, 7, 2.2508202), (11, 9, 0.49912065)]
[(8, 7, 0.9851221980588887), (11, 9, 1.0148778019411113)]
[(8, 7, 2.2508202), (11, 9, 0.49912065), (21, 5, 0.51507616)]
[(8, 7, 0.6744800448818367), (11, 9, 0.6936017772688059), (21, 5, 0.6319181778493574)]
[(8, 7, 2.2508202), (11, 9, 0.49912065), (21, 5, 0.51507616)]
[(8, 7, 0.6744800448818367), (11, 9, 0.6936017772688059), (21, 5, 0.6319181778493574)]
[(8, 7, 2.2508202), (11, 9, 0.49912065), (21, 5, 0.51507616), (28, 3, 0.5170754)]
[(8, 7, 0.5319204526870114), (11, 9, 0.5460169866657945), (21, 5, 0.49564661629542406), (28, 3, 0.4264159443517701)]
[(8, 7, 2.2508202), (11, 9, 0.49912065), (21, 5, 0.51507616), (28, 3, 0.5170754), (30, 1, 0.519131)]
[(8, 7, 0.4470683741420237), (11, 9, 0.45831489581031515), (21, 5, 0.4149254113794831), (28, 3, 0.356021635319663), (30, 1, 0.323669683348515)]
[(8, 7, 2.2508202), (11, 9, 0.49912065), (21, 5, 0.51507616), (28, 3, 0.5170754), (30, 1, 0.519131)]
[(8, 7, 0.44706837414202

KeyboardInterrupt: 

In [10]:
parameters[0]

Parameter containing:
tensor([[ 0.1045,  0.4002, -0.1971,  0.1176, -0.1599, -0.0181, -0.1915]],
       requires_grad=True)

In [12]:
2.142857142857143 * 0.7

1.4999999999999998

In [45]:
model._modules.items()

odict_items([('input_layer', Sequential(
  (0): Linear(in_features=32, out_features=64, bias=True)
  (1): ReLU()
)), ('output_layer', Sequential(
  (0): Linear(in_features=64, out_features=1, bias=True)
))])

# Training

In [150]:


INPUT_SEQ_LEN = 1
TARGET_SEQ_LEN = 1
HIDDEN_SIZE = 32
# TODO fix BATCH_SIZE > 1 is bugged.
BATCH_SIZE = 32
TEACHER_FORCING_PROB = 0.5
LEARNING_RATE = 1e-3
PRINT_LOSS_EVERY = 100
model = SimpleEnd2End()
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.33, 0.66
                                                                                                                    ], seed=0)
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=2)

parameters = (list(model.parameters()))

optimizer = optim.SGD(parameters
                       ,lr=LEARNING_RATE)
criterion = nn.MSELoss()
avg_loss_window = np.zeros(PRINT_LOSS_EVERY)
avg_loss_window_idx = 0

for batch_idx, data in enumerate(dataloader):
    reference_cell_input =  data[0].view(BATCH_SIZE, 1)
    reference_cell_present_input = data[1].view(BATCH_SIZE, 1)
    neighbourhood_cell_rel_input = data[2].view(BATCH_SIZE, 12)
    neighbourhood_cell_load_input = data[3].view(BATCH_SIZE, 6)
    reference_cell_target = data[4].view(BATCH_SIZE, 1)
    reference_cell_present_target = data[5].view(BATCH_SIZE, 1)
    neighbourhood_cell_rel_target = data[6].view(BATCH_SIZE, 12)
    #print(reference_cell_input) 
    
    if batch_idx == 0:
        writer.add_graph(model, (reference_cell_input, reference_cell_present_input,
    neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_rel_target), True)
    
    # Manual input
    optimizer.zero_grad()
    #print(reference_cell_input.shape)
    output = model(reference_cell_input, reference_cell_present_input,
    neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_rel_target)
    loss = criterion(output, reference_cell_target[:, 0])
    #print("target:{}".format(reference_cell_target[:, 0]))

    loss.backward()
    optimizer.step()
    avg_loss_window[avg_loss_window_idx] = loss.data
    #writer.add_histogram('parameter_0', parameters[0].data.numpy(), batch_idx)
    #writer.add_histogram('parameter_1', parameters[1].data.numpy(), batch_idx)
    #writer.add_histogram('parameter_2', parameters[2].data.numpy(), batch_idx)
    #writer.add_histogram('parameter_3', parameters[3].data.numpy(), batch_idx)
    if avg_loss_window_idx >= PRINT_LOSS_EVERY - 1:
        writer.add_scalar('loss', loss.data.numpy(), batch_idx)
        plot_parameters(model)
        avg_loss_window_idx = 0
        #print(avg_loss_window.mean())
        #print(list(decoder.parameters())[0].grad.data)
        #for param in parameters:
        #    print(param.shape)
        #    print(param.grad.data)
        # print("decoder_output {}".format(decoder_output))
        #print(decoder_output - reference_cell_target[:, seq_idx])
    else:
        avg_loss_window_idx += 1         

graph(%0 : Float(32, 1)
      %1 : Float(32, 1)
      %2 : Float(32, 12)
      %3 : Float(32, 6)
      %4 : Float(32, 12)
      %5 : Float(64, 32)
      %6 : Float(64)
      %7 : Float(1, 64)
      %8 : Float(1)) {
  %9 : Float(32, 2) = onnx::Concat[axis=1](%0, %1), scope: SimpleEnd2End
  %10 : Float(32, 14) = onnx::Concat[axis=1](%9, %2), scope: SimpleEnd2End
  %11 : Float(32, 20) = onnx::Concat[axis=1](%10, %3), scope: SimpleEnd2End
  %12 : Float(32, 32) = onnx::Concat[axis=1](%11, %4), scope: SimpleEnd2End
  %13 : Float(32!, 64!) = onnx::Transpose[perm=[1, 0]](%5), scope: SimpleEnd2End/Sequential[input_layer]/Linear[0]
  %14 : Float(32, 64) = onnx::Gemm[alpha=1, beta=1](%12, %13, %6), scope: SimpleEnd2End/Sequential[input_layer]/Linear[0]
  %15 : Float(32, 64) = onnx::Relu(%14), scope: SimpleEnd2End/Sequential[input_layer]/ReLU[1]
  %16 : Float(64!, 1!) = onnx::Transpose[perm=[1, 0]](%7), scope: SimpleEnd2End/Sequential[output_layer]/Linear[0]
  %17 : Float(32, 1) = onnx::Gemm[alpha

KeyboardInterrupt: 

In [1]:
DynamicTopologyModel(neighbourhood_hidden_size=32, neighbourhood_cell_count=6,
                     neighbourhood_output_size=16, gru_layers=2, teacher_forcing_probability=1.0)

NameError: name 'DynamicTopologyModel' is not defined

In [14]:
parameters[3]

Parameter containing:
tensor([0.3525], requires_grad=True)

In [8]:
parameters

[Parameter containing:
 tensor([[ 0.1045,  0.4002, -0.1971,  0.1176, -0.1599, -0.0181, -0.1915]],
        requires_grad=True), Parameter containing:
 tensor([0.5222], requires_grad=True)]

In [13]:
INPUT_SEQ_LEN = 1
TARGET_SEQ_LEN = 1
HIDDEN_SIZE = 32
# TODO fix BATCH_SIZE > 1 is bugged.
BATCH_SIZE = 32 
TEACHER_FORCING_PROB = 0.5
LEARNING_RATE = 1e-3
PRINT_LOSS_EVERY = 100
neighbourhood_fully_connected = NeighbourhoodFullyConnected(2, 1, 16, 16)
neighbourhood_fully_connected_decoder = NeighbourhoodFullyConnectedDecoder(2, 16, 16)
encoder = EncoderRNN(1 + 1 + 16, HIDDEN_SIZE)
decoder = DecoderRNN(18, HIDDEN_SIZE)
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.33, 0.66
                                                                                                                    ])
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=1)

parameters = (list(neighbourhood_fully_connected.parameters()) + 
                       list(neighbourhood_fully_connected_decoder.parameters()) +
                       list(encoder.parameters()) +
                       list(decoder.parameters()))
#parameters = (list(neighbourhood_fully_connected.parameters()) + 
#                       list(encoder.parameters()))

optimizer = optim.Adam(parameters
                       , lr=LEARNING_RATE)
criterion = nn.MSELoss()
avg_loss_window = np.zeros(PRINT_LOSS_EVERY)
avg_loss_window_idx = 0

for batch_idx, data in enumerate(dataloader):
    reference_cell_input =  data[0]
    reference_cell_present_input = data[1]
    neighbourhood_cell_rel_input = data[2]
    neighbourhood_cell_load_input = data[3]
    reference_cell_target = data[4]
    reference_cell_present_target = data[5]
    neighbourhood_cell_rel_target = data[6]
    optimizer.zero_grad()
    for seq_idx in range(reference_cell_input.size(1)):
        neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)
        for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
            output = neighbourhood_fully_connected(neighbourhood_cell_rel_input[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2),
                                          neighbourhood_cell_load_input[:,seq_idx, neighbourhood_cell_idx].view(BATCH_SIZE, 1))
            neighbourhood_influence = torch.add(neighbourhood_influence, output)
        # Concatonate neighbourhood with reference cell time series.
        writer.add_graph(neighbourhood_fully_connected, (neighbourhood_cell_rel_input[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2),
                                          neighbourhood_cell_load_input[:,seq_idx, neighbourhood_cell_idx].view(BATCH_SIZE, 1)), True)
        cat_ref_input = (torch.cat((reference_cell_input[:, seq_idx].view(BATCH_SIZE, 1),
                                            reference_cell_present_input[:, seq_idx].view(BATCH_SIZE, 1)), dim=1))
        encoder_input = torch.cat((cat_ref_input , neighbourhood_influence), dim=1).view(BATCH_SIZE, 1, 1 + 1+ 16)
        if seq_idx==0:
            encoder_hidden = torch.zeros(2, BATCH_SIZE, HIDDEN_SIZE, device=device)
        encoder_output, encoder_hidden = encoder(encoder_input, encoder_hidden)
        #if seq_idx==0:
            #loss = criterion(torch.rand(32, 128), encoder_output)
        #else:
            ##########################
            # DELETE ME!
            #########################
            #loss += criterion(torch.rand(32, 128), encoder_output)
    #avg_loss_window[avg_loss_window_idx] = loss.data

       

    #loss.backward()
    
    #if avg_loss_window_idx >= PRINT_LOSS_EVERY - 1:
        #avg_loss_window_idx = 0
        #print(avg_loss_window.mean())
        #for param in parameters:
            #print(param.shape)
            #print(param.grad.data)
    #else:
        #avg_loss_window_idx += 1  
    #optimizer.step()

    ##########################
    # DELETE ME!
    #########################
    
    for seq_idx in range(reference_cell_target.size(1)):
        neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)

        for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
            output = neighbourhood_fully_connected_decoder(neighbourhood_cell_rel_target[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2))
            neighbourhood_influence = torch.add(neighbourhood_influence, output)
        #print(neighbourhood_influence)
        # Feed in the last input in the sequence into the decoder and use the last encoder hidden state as the initial hidden state.
        #print(reference_cell_input[:, -1])
        #print(reference_cell_present_input[:, -1])
        if seq_idx==0:
            decoder_output, decoder_hidden = decoder(reference_cell_input[:, -1].view(BATCH_SIZE, 1),
                                                     reference_cell_present_input[:, -1].view(BATCH_SIZE, 1),
                                                     neighbourhood_influence.view(BATCH_SIZE, 16),
                                                     encoder_hidden)
            loss = criterion(decoder_output, reference_cell_target[:, seq_idx])
        if random.random() > TEACHER_FORCING_PROB:
            decoder_output, decoder_hidden = decoder(reference_cell_target[:, seq_idx - 1].view(BATCH_SIZE, 1),
                                                     reference_cell_present_target[:, seq_idx -1].view(BATCH_SIZE, 1),
                                                     neighbourhood_influence.view(BATCH_SIZE, 16),
                                                     decoder_hidden)
        else:
            decoder_output, decoder_hidden = decoder(decoder_output.view(BATCH_SIZE, 1),
                                                     reference_cell_present_target[:, seq_idx -1].view(BATCH_SIZE, 1),
                                                     neighbourhood_influence.view(BATCH_SIZE, 16),
                                                     decoder_hidden)
            loss += criterion(decoder_output, reference_cell_target[:, seq_idx])

            
    loss.backward()
    avg_loss_window[avg_loss_window_idx] = loss.data
    writer.add_scalar('loss', avg_loss_window.mean(), batch_idx)
    writer.add_histogram('parameter_0', parameters[0].data.numpy(), batch_idx)
    if avg_loss_window_idx >= PRINT_LOSS_EVERY - 1:
        
        avg_loss_window_idx = 0
        print(avg_loss_window.mean())
        #for param in parameters:
            #print(param.shape)
            #print(param.grad.data)
        #print(decoder_output - reference_cell_target[:, seq_idx])
    else:
        avg_loss_window_idx += 1         
    optimizer.step()

torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
graph(%input.1 : Float(32!, 2)
      %input.3 : Float(32!, 1)
      %2 : Float(32, 2)
      %3 : Float(32)
      %4 : Float(32, 1)
      %5 : Float(32)
      %6 : Float(16, 32)
      %7 : Float(16)
      %8 : Float(16, 32)
      %9 : Float(16)) {
  %10 : Float(2!, 32!) = onnx::Transpose[perm=[1, 0]](%2), scope: NeighbourhoodFullyConnected/Sequential[neighbourhood_rel_input_layer]/Linear[0]
  %11 : Float(32, 3

/Users/d854956/personal/dynamic_topology/models.py:57: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  print(neighbourhood_rel_hidden_val.shape)
/Users/d854956/personal/dynamic_topology/models.py:59: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  print(neighbourhood_load_hidden_val.shape)
/Users/d854956/personal/dynamic_topology/models.py:61: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that

torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 16])
torch.Size([32, 16])
graph(%input.1 : Float(32!, 2)
      %input.3 : Float(32!, 1)
      %2 : Float(32, 2)
      %3 : Float(32)
      %4 : Float(32, 1)
      %5 : Float(32)
      %6 : Float(16, 32)
      %7 : Float(16)
      %8 : Float(16, 32)
      %9 : Float(16)) {
  %10 : Float(2!, 32!) = onnx::Transpose[perm=[1, 0]](%2), scope: NeighbourhoodFullyConnected/Sequential[neighbourhood_rel_input_layer]/Linear[0]
  %11 : Float(32, 3

KeyboardInterrupt: 

array([[-0.50105333, -0.03621238],
       [-0.6716445 ,  0.2170561 ],
       [-0.07053212, -0.21118125],
       [ 0.35602853,  0.3363651 ],
       [-0.31223166,  0.35850143],
       [-0.7075539 , -0.11176805],
       [-0.03756676, -0.18710963],
       [ 0.11037824,  0.351485  ],
       [-0.48305982,  0.5724187 ],
       [ 0.62445164,  0.148751  ],
       [ 0.3769189 , -0.05743391],
       [-0.16877972, -0.1388054 ],
       [-0.3683863 ,  0.6243406 ],
       [ 0.24089734, -0.33954647],
       [ 0.3014789 , -0.16865112],
       [-0.27933788, -0.1143957 ]], dtype=float32)

In [17]:
type(loss.grad)

NoneType

In [14]:
list(encoder.parameters())[0]


Parameter containing:
tensor([[ 0.1148,  0.1018, -0.2019,  0.0415, -0.0409, -0.2270,  0.2182,  0.1528,
         -0.0238,  0.0319, -0.0558, -0.0170,  0.0188, -0.0592,  0.0674,  0.1079,
         -0.0682,  0.1211],
        [-0.2490,  0.1737,  0.1887, -0.0509,  0.0821, -0.1362, -0.1827, -0.0332,
         -0.0813,  0.1133, -0.1793, -0.1379,  0.0746,  0.0311, -0.0412,  0.0491,
         -0.0063, -0.1207],
        [-0.0639, -0.2160, -0.0790, -0.1887, -0.2419,  0.0460, -0.1359,  0.0479,
         -0.2079,  0.0567, -0.2594, -0.2213,  0.0427,  0.0444, -0.0648,  0.1961,
         -0.1447,  0.0390],
        [-0.0062, -0.0581,  0.1298, -0.0125,  0.1165, -0.1581, -0.1394, -0.0580,
         -0.1481,  0.1173, -0.1226, -0.0205, -0.0550, -0.1158, -0.1596, -0.1481,
         -0.1614, -0.0857],
        [ 0.1030, -0.2071, -0.1594, -0.1416, -0.2311, -0.1790, -0.1107, -0.0542,
          0.1006,  0.2105, -0.1977, -0.2460,  0.1470, -0.1744,  0.1268, -0.1630,
          0.0480,  0.1013],
        [-0.0702, -0.1749, -

# Legacy Visualisation

In [12]:
screen_draw_idx = 0
cells_seq = [list() for i in range(SCREEN_COUNT)]

def add_cell_if_not_exists(new_cell, cells):
    cell_already_exists = False

    for i, cell in enumerate(cells):
        if cell[0] == new_cell[0] and cell[1] == new_cell[1]:
            cell_already_exists = True
            break
    if not cell_already_exists:
        cells.append(new_cell)
    return cells

def remove_cell_if_exists(remove_cell, cells):
    for i, cell in enumerate(cells):
        if cell[0] == remove_cell[0] and cell[1] == remove_cell[1]:
            cells.pop(i)
            break
    return cells

while True:
    
    
    screen_draw = pygame.Surface((PIXELS, PIXELS))
    
    #print(cells_seq)
    cells = cells_seq[screen_draw_idx]
    if len(cells) > 0:
        cell_loads = load.calculate_cell_load(cells, loads)
        #print(cell_loads)
        #print(loads)



        # Update screen
        for cell in cell_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )
        

    e = pygame.event.poll()


    if e.type == pygame.QUIT:
        raise StopIteration
    if e.type == pygame.MOUSEBUTTONDOWN:
        leftclick, middleclick, rightclick = pygame.mouse.get_pressed()
        x, y = e.pos
        x_cell= int(x / PIXELS_PER_BLOCK)
        y_cell = int(y / PIXELS_PER_BLOCK)
        if leftclick:
            cells = add_cell_if_not_exists((x_cell, y_cell), cells)
        elif rightclick:
            cells = remove_cell_if_exists((x_cell, y_cell), cells)            
        draw_on = True
        # Update cells
        cells_seq[screen_draw_idx] = cells
    
    if e.type == pygame.KEYDOWN:
        if e.key == pygame.K_SPACE:
            if freeze:
                freeze = False
            else:
                freeze = True
        if e.key ==  pygame.K_LEFT:
            if screen_draw_idx > 0:
                screen_draw_idx -= 1
        if e.key ==  pygame.K_RIGHT:
            if screen_draw_idx < SCREEN_COUNT - 1:
                screen_draw_idx += 1
    if e.type == pygame.MOUSEBUTTONUP:
        draw_on = False
    pygame.display.flip()
    target_screen = pygame.surfarray.array3d(screen_draw)
    display_img = block_img + target_screen
    new_surf = pygame.pixelcopy.make_surface(display_img.astype(np.uint8))
    screen.blit(new_surf, (0, 0))

KeyboardInterrupt: 

In [49]:
class SimpleFF1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleFF1, self).__init__()
        self.input_size = input_size
        self.input_layer = nn.Sequential(nn.Linear(input_size, hidden_size), nn.LeakyReLU())
        self.output_layer = nn.Sequential(nn.Linear(hidden_size, 1))

    def forward(self, input):
        output = self.input_layer(input)
        output = self.output_layer(output)
        return output

In [50]:
class MultipleFF(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MultipleFF, self).__init__()
        self.model0 = SimpleFF1(input_size, hidden_size)
        self.model1 = SimpleFF1(1, hidden_size)
        self.output_activation = torch.nn.Sigmoid()

    def forward(self, input):
        output = self.model0(input)
        output = self.model1(output)
        output = self.output_activation(output)
        return output

In [87]:
simpleff = MultipleFF(16, 32)

In [78]:
#input_values = torch.autograd.Variable(torch.ones(16), requires_grad=True)
input_values = torch.ones(16)

In [79]:
simpleff(input_values)

tensor([0.0344], grad_fn=<AddBackward0>)

In [80]:
writer.add_graph(simpleff, input_values, True)

graph(%input.1 : Float(16)
      %1 : Float(32, 16)
      %2 : Float(32)
      %3 : Float(1, 32)
      %4 : Float(1)) {
  %5 : Float(16!, 32!) = onnx::Transpose[perm=[1, 0]](%1), scope: SimpleFF1/Sequential[input_layer]/Linear[0]
  %6 : Float(32) = onnx::MatMul(%input.1, %5), scope: SimpleFF1/Sequential[input_layer]/Linear[0]
  %7 : Float(32) = onnx::Add(%6, %2), scope: SimpleFF1/Sequential[input_layer]/Linear[0]
  %8 : Float(32) = onnx::LeakyRelu[alpha=0.01](%7), scope: SimpleFF1/Sequential[input_layer]/LeakyReLU[1]
  %9 : Float(32!, 1!) = onnx::Transpose[perm=[1, 0]](%3), scope: SimpleFF1/Sequential[output_layer]/Linear[0]
  %10 : Float(1) = onnx::MatMul(%8, %9), scope: SimpleFF1/Sequential[output_layer]/Linear[0]
  %11 : Float(1) = onnx::Add(%10, %4), scope: SimpleFF1/Sequential[output_layer]/Linear[0]
  return (%11);
}



In [81]:
for i, k, v in enumerate(simpleff._modules.items()):
    print(type(v))
    print(i)
    print(k)
    print(v)

ValueError: not enough values to unpack (expected 3, got 2)

In [84]:
for i, kv in enumerate(simpleff._modules.items()):
    k, v = kv 
    scope = k
    weights = v.weight()
    print("{}: {}".format(scope, list(weights)))
    print(type(v))
    print(i)
    print(k)
    print(v)

AttributeError: 'Sequential' object has no attribute 'weight'

In [83]:
simpleff.parameters

<bound method Module.parameters of SimpleFF1(
  (input_layer): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=32, out_features=1, bias=True)
  )
)>

In [148]:
def plot_parameters(model):
    for k, v in model.state_dict().items():
        shape = v.shape
        # Don't do this for single weights or biases
        if np.any(np.array(shape) > 1):
            mean = torch.mean(v)
            std_dev = torch.std(v)
            maximum = torch.max(v)
            minimum = torch.min(v)
            writer.add_scalars("layer_{}_{} ".format(k, shape), {"mean": mean,
                                                                    "std_dev": std_dev,
                                                                    "max": maximum,
                                                                    "min": minimum}, batch_idx)
            writer.add_histogram("layer_{}_{}".format(k, shape), v, batch_idx)
            
        else:
            writer.add_scalar("layer_{}_{}".format(k, shape), v.data, batch_idx)

In [141]:
np.any(np.array([1, 1]) > 1)

False

In [142]:
a = torch.all(torch.Tensor([1, 3 ,5]))

RuntimeError: _th_all is not implemented for type torch.FloatTensor

In [143]:
plot_parameters(simpleff)